# Experimentando difententes métodos com OTSU para calcular o LBP

> a imagem é disposta como uma matriz no qual os pixels variam de 0 a 1.

- menor: pixels abaixo do limiar de OTSU se tornam 0;
- maior: pixels acima ou igual ao limiar de OTSU se tornam 1;
- ambos (_default_): pixels abaixo do limiar de OTSU se tornam 0, os demais se tornam 1;

# Importando bibliotecas

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from zipfile import ZipFile
from skimage.io import imread
from img_edit import preprocessing_img

from skimage.filters import threshold_otsu
from skimage.feature import local_binary_pattern

# Parâmetros do LBP
METHOD = 'nri_uniform'
RADIUS = 1
N_POINTS = [4, 8]

ZIP_PATH = 'patologias_toras.zip'
TARGET_NAMES = {'normal': 0,
                'broca':1,
                'fissura': 2,
                'musgo': 3}

plt.gray()

In [6]:
path_dict = {name: [] for name in TARGET_NAMES}

with ZipFile(ZIP_PATH) as zip_file:
  for path in zip_file.namelist():
    # a pasta de cada amostra é o segundo elemento do split
    folder = path.split('/')[1]
    # Separando amostras por classe
    if '__MAC' not in path \
        and '.JPG' in path \
        and folder in TARGET_NAMES:
      path_dict[folder].append(f'./{ZIP_PATH}/{path}')

# Mostrando quantidade de amostras por classe
qtd_amostras_p_classe = []
for target in path_dict:
  n_amostras = len(path_dict[target])
  print(f'{target}: {n_amostras} amostras')

  qtd_amostras_p_classe.append(n_amostras)

N_MIN_DE_AMOSTRAS = min(qtd_amostras_p_classe)

normal: 107 amostras
broca: 90 amostras
fissura: 109 amostras
musgo: 111 amostras


In [7]:
def menor(arr, limiar) -> np.ndarray:
  img_otsu = arr.copy()
  img_otsu[arr<limiar] = 0
  return img_otsu

def maior(arr, limiar) -> np.ndarray:
  img_otsu = arr.copy()
  img_otsu[arr>=limiar] = 1
  return img_otsu

def ambos(arr, limiar) -> np.ndarray:
  return arr >= limiar

In [8]:
for points, n_bins in zip(N_POINTS, [17, 60]):
  print(f'\ncalculando LBP com {points} pontos')
  target_list = []
  angle_conf = []
  crop_conf = []
  color_list = []
  func_list = [(ambos, []),
               (menor, []),
               (maior, [])]

  for target_name in TARGET_NAMES:
    print(f'classe {target_name}')
    path_list = path_dict[target_name]
    path_list_slice = path_list[:N_MIN_DE_AMOSTRAS]

    for path in path_list_slice:
      img = imread(path)
      img_name = path.split('/')[-1].lower()

      img_prepro = preprocessing_img(img, return_metadata=True)
      new_img = img_prepro['img']

      if not img_prepro['conf_angle']:
        print(f'{img_name} rotacionada em {round(img_prepro["angle"], 3)} com moda < 3')
      
      if not img_prepro['conf_crop']:
        print(f'{img_name} não foi cortada.')
      
      # Salvando imagens
      # plt.imsave(f'./imgs/prepro_P{points}/{target_name}-{img_name}', arr=new_img)

      target_list.append(TARGET_NAMES[target_name])
      color_list.append(img_prepro['color'])
      angle_conf.append(img_prepro['conf_angle'])
      crop_conf.append(img_prepro['conf_crop'])
      # Gerando arquivos csv com LBP

      img_resolution = new_img.shape[0] * new_img.shape[1]
      for func, amostras in func_list:
        img_otsu = func(new_img, threshold_otsu(new_img))

        lbp_flat = local_binary_pattern(image=img_otsu,
                                        P=points,
                                        R=RADIUS,
                                        method=METHOD).flatten()
        
        arr, _ = np.histogram(lbp_flat, bins=range(n_bins), density=False)
        arr = arr / img_resolution

        amostras.append(arr)

  for func, amostras in func_list:
    data = pd.DataFrame(amostras)
    data['target'] = target_list
    data.to_csv(f'./csvs/prepro_5-{func.__name__}-P{points}.csv', index=False)

  d = {'targets':target_list,
      'colors':color_list,
      'angle_conf':angle_conf,
      'crop_conf':crop_conf}
  df = pd.DataFrame(data=d, dtype=int)


calculando LBP com 4 pontos
classe normal


<string>:6: RuntimeWarning: Converting input from bool to <class 'numpy.uint8'> for compatibility.


toras515.jpg rotacionada em 0.5 com moda < 3
toras771.jpg rotacionada em 36.5 com moda < 3
toras717.jpg rotacionada em 0.0 com moda < 3
toras713.jpg rotacionada em 9.5 com moda < 3
toras336.jpg rotacionada em 10.0 com moda < 3
toras319.jpg rotacionada em 41.0 com moda < 3
toras110.jpg rotacionada em 0.0 com moda < 3
toras604.jpg rotacionada em 0.0 com moda < 3
toras20.jpg rotacionada em -44.0 com moda < 3
classe broca
toras1874.jpg rotacionada em 0.0 com moda < 3
toras1745.jpg não foi cortada.
toras1412.jpg rotacionada em -12.0 com moda < 3
toras1408.jpg rotacionada em -43.0 com moda < 3
toras1483.jpg rotacionada em 43.5 com moda < 3
toras573.jpg rotacionada em 0.0 com moda < 3
toras332.jpg rotacionada em 5.5 com moda < 3
toras496.jpg rotacionada em -10.5 com moda < 3
classe fissura
toras884.jpg rotacionada em 0.0 com moda < 3
toras731.jpg rotacionada em 34.0 com moda < 3
toras718.jpg rotacionada em 43.5 com moda < 3
toras692.jpg rotacionada em 0.0 com moda < 3
toras206.jpg rotacionada